<a href="https://colab.research.google.com/github/BaseKan/optimisation_workshop/blob/main/dash_app_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
!curl -L -c cookies.txt 'https://docs.google.com/uc?export=download&id=1jqIT0Oimj1sXUfNj071u7l4OSU93Fc_y' | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1/p' > confirm.txt    
!curl -L -b cookies.txt -o 'app.zip' 'https://docs.google.com/uc?export=download&id=1jqIT0Oimj1sXUfNj071u7l4OSU93Fc_y&confirm='$(<confirm.txt)
!unzip app.zip
!rm -f confirm.txt cookies.txt app.zip

In [ ]:
!curl -L -c cookies.txt 'https://docs.google.com/uc?export=download&id=1PQWIANcav-KpiO2Ju8wIKfeB8Gpwizqs' | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1/p' > confirm.txt    
!curl -L -b cookies.txt -o 'data.zip' 'https://docs.google.com/uc?export=download&id=1PQWIANcav-KpiO2Ju8wIKfeB8Gpwizqs&confirm='$(<confirm.txt)
!unzip data.zip
!rm -f confirm.txt cookies.txt data.zip

In [ ]:
!pip install jupyter-dash
!pip install dash-bootstrap-components

# EnergieNL App

In [ ]:
import pandas as pd
import os
import numpy as np

def load_data_from_file(datapath, filename):
    path = os.path.join(datapath,filename)
    return pd.read_csv(path).assign(jaar =  filename.split('_')[2][:4],
                                                         dso = filename.split('_')[0])

def load_timeseries_data(basepath='data', type = 'Electricity'):
    datapath = os.path.join(basepath,type)
    files = os.listdir(datapath)
    files = [filename for filename in files if int(filename.split('_')[2][:4])>= 2017]
    dsos = ['liander','enexis','stedin']
    files = [filename for filename in files if any(dso in filename for dso in dsos)]
    df = pd.concat([load_data_from_file(datapath, file) for file in files])
    return df

def calculate_top_ten(df):
    df = df.assign(Verbruik = df['annual_consume']).filter(['Verbruik','city'])
    df = (
        df.filter(['city','Verbruik']).
            groupby(['city'])
            .agg(np.sum)
            .sort_values('Verbruik',ascending=False)
            .reset_index()
            .iloc[:10,:]
    )
    df.columns = ['Stad', 'Jaarverbruik (MWh)']
    return df

def calculate_timeseries(df_elec, df_gas, cities):
    df_elec = (
        df_elec.query(f'city in {cities}')
            .filter(['city','jaar','annual_consume','num_connections'])
            .groupby(['city','jaar'])
            .agg(np.sum)
    )
    df_elec.columns = ['E','num_connections']
    df_gas = (
        df_gas.query(f'city in {cities}')
            .filter(['city','jaar', 'annual_consume'])
            .groupby(['city', 'jaar'])
            .agg(np.sum)
    )
    df_gas.columns = ['G']
    df = df_gas.join(df_elec).reset_index()
    df = (
        df.assign(Gemiddeld_verbruik = (df['E']*3.6 + df['G']*35.17)/df['num_connections'])
        .filter(['city','jaar','Gemiddeld_verbruik'])
    )
    df.columns = ['Stad','Jaar','Gemiddeld_verbruik']
    return df

In [ ]:
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_bootstrap_components as dbc
import dash_html_components as html
from dash.dependencies import Input, Output, State

import plotly.express as px

import page_jaarverbruik
import page_tijdreeks
import time

external_stylesheets = ["https://bootswatch.com/4/flatly/bootstrap.css"]

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

overhead_bar = dbc.NavbarSimple(
    children = [dbc.NavItem(dbc.NavLink("Stedelijk Jaarverbruik", href='/jaarverbruik', active="exact")),
                dbc.NavItem(dbc.NavLink("Verbruik over de jaren", href='/tijdreeks', active="exact"))],
    brand='EnergieNL v0.1.0',
    color='primary',
    dark=True
)

content = html.Div(id='page-content', children=[])

app.layout = html.Div([
    dcc.Location(id='url'),
    overhead_bar,
    content
])

@app.callback(Output('page-content','children'),
              [Input('url','pathname')])

def render_page_content(pathname):
    if pathname == "/" or pathname == '/jaarverbruik':
        return page_jaarverbruik.content

    elif pathname == '/tijdreeks':
        return page_tijdreeks.content

    else:
        return dbc.Jumbotron(
        [
            html.H1("404: Not found", className="text-danger"),
            html.Hr(),
            html.P(f"The pathname {pathname} was not recognised..."),
        ]
    )

@app.callback(Output("dso-table", "data"),
              Output('calc-box1', 'children'),
              Input("submit-table-data", 'n_clicks'),
              State("year-box", "value"),
              State("dso-box", "value"))

def update_dso_table(n_clicks, year, dso):
    start = time.time()
    df = load_data_from_file(datapath='data/Electricity',filename=dso+'_electricity_'+str(year)+'.csv')
    df = calculate_top_ten(df=df)
    end = time.time()
    rekentijd_str = str(round(end-start,3))+' seconden'
    return df.to_dict('records'), rekentijd_str

@app.callback(Output("timeseries-figure", "figure"),
              Output('calc-box2', 'children'),
              Input("submit-figure-data", 'n_clicks'),
              State("city-box", "value"))

def update_timeseries_figure(n_clicks, cities):
    start = time.time()
    df_elec = load_timeseries_data()
    df_gas = load_timeseries_data(type='Gas')
    df = calculate_timeseries(df_elec=df_elec,df_gas=df_gas, cities=cities)
    fig = px.line(df.sort_values('Jaar'), x='Jaar', y='Gemiddeld_verbruik', color='Stad')
    end = time.time()
    rekentijd_str = str(round(end-start,3))+' seconden'
    return fig, rekentijd_str



app.run_server(mode='external', debug=True)
